Lab Exercises 3 and 4

Sidaray M S

1RVU23CSE453

In [ ]:
!pip install groq
import os
from groq import Groq

# Install the groq library
!pip install groq

# 1. SETUP API KEY
# Replace 'YOUR_GROQ_API_KEY' with your actual key
os.environ["GROQ_API_KEY"] = "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"

# 2. INITIALIZE CLIENT
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Define the model to use (Fast and Efficient)
MODEL_ID = "llama-3.1-8b-instant"

print(f"Setup Complete. Using Groq model: {MODEL_ID}")

Setup Complete. Using Groq model: llama-3.1-8b-instant


In [ ]:
def classify_sentiment(text, strategy="zero-shot"):
    messages = []

    if strategy == "zero-shot":
        prompt = f"Classify the sentiment of this text (Positive/Negative/Neutral): '{text}'"
        messages = [{"role": "user", "content": prompt}]

    elif strategy == "one-shot":
        # We provide an example interaction in the history
        messages = [
            {"role": "user", "content": "Text: 'I loved the movie!'"},
            {"role": "assistant", "content": "Sentiment: Positive"},
            {"role": "user", "content": f"Text: '{text}'"}
        ]

    elif strategy == "few-shot":
        # Multiple examples
        messages = [
            {"role": "user", "content": "Text: 'The food was cold.'"},
            {"role": "assistant", "content": "Sentiment: Negative"},
            {"role": "user", "content": "Text: 'I loved the movie!'"},
            {"role": "assistant", "content": "Sentiment: Positive"},
            {"role": "user", "content": f"Text: '{text}'"}
        ]

    completion = client.chat.completions.create(
        model=MODEL_ID,
        messages=messages,
        temperature=0,
        max_tokens=10
    )
    return completion.choices[0].message.content.strip()

# Test the function
print(f"Zero-Shot: {classify_sentiment('The service was slow.', 'zero-shot')}")
print(f"Few-Shot:  {classify_sentiment('The service was slow.', 'few-shot')}")

Zero-Shot: The sentiment of this text is Negative. The word
Few-Shot:  Sentiment: Negative


In [ ]:
def rgc_prompt(user_query):
    # RGC Components
    role = "You are a senior Python Data Science Instructor."
    context = "The user is a beginner student struggling with pandas dataframes."
    goal = "Explain concepts simply using food metaphors."

    # We use the 'system' role for the Persona/RGC
    completion = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {
                "role": "system",
                "content": f"Role: {role}\nContext: {context}\nGoal: {goal}"
            },
            {
                "role": "user",
                "content": user_query
            }
        ]
    )
    return completion.choices[0].message.content

print(rgc_prompt("How do I merge two dataframes?"))

Imagine you're running a popular food truck with two different menus: Menu A and Menu B. Each menu is like a separate pandas DataFrame. You want to combine Menu A and Menu B into one big menu that you can show your customers. This is similar to merging two dataframes.

Here are the different ways to merge two dataframes, just like how you can combine your menus:

1. **Inner Merge**: This is like serving dishes only if they're available on both menus. You're matching up items like "Burger" on Menu A with "Burger" on Menu B.
```python
import pandas as pd

menu_a = pd.DataFrame({
    'Item': ['Burger', 'Salad', 'Sandwich'],
    'Price': [10, 8, 12]
})

menu_b = pd.DataFrame({
    'Item': ['Burger', 'Fries', 'Salad'],
    'Price': [10, 5, 8]
})

merged_menu = pd.merge(menu_a, menu_b, on='Item')
print(merged_menu)
```

Output:
```
      Item  Price_x  Price_y
0    Burger       10       10
1    Salad        8        8
```

2. **Left Merge**: This is like serving items from Menu A, even if th

In [ ]:
def math_reasoning_table(problem):
    prompt = f"""
    Solve this logic puzzle: {problem}

    Instructions:
    1. Practice Chain of Thought: Break down the steps logically first.
    2. Format the output in a Markdown Table with columns: [Step #, Reasoning, Calculation].
    """

    completion = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content

puzzle = "If I have 5 apples, eat 2, buy 3 more, and then drop half of what I have, how many do I have?"
print(math_reasoning_table(puzzle))

**Logic Puzzle Solution**

We will break down the problem into a series of logical steps and then perform calculations.

### Step 1: Initial Condition
| Step # | Reasoning                 | Calculation        |
| :---- | :----------------------- | :----------------- |
| 1     | Initial number of apples  | 5 apples           |

### Step 2: Eat 2 Apples
| Step # | Reasoning                 | Calculation        |
| :---- | :----------------------- | :----------------- |
| 2     | I have eaten 2 apples     | 5 apples - 2 apples = 3 apples |

### Step 3: Buy 3 More Apples
| Step # | Reasoning                 | Calculation        |
| :---- | :----------------------- | :----------------- |
| 3     | I bought 3 new apples     | 3 apples + 3 apples = 6 apples |

### Step 4: Drop Half of What I Have
| Step # | Reasoning                 | Calculation        |
| :---- | :----------------------- | :----------------- |
| 4     | Half of 6 apples is 3     | 6 apples - 3 apples = 3 apples |

So, after

In [ ]:
def fill_in_the_blank(sentence):
    prompt = f"""
    Complete the following sentence by filling in the missing word marked as [BLANK].
    Only provide the word, nothing else.

    Sentence: {sentence}
    """

    completion = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content

print(fill_in_the_blank("The capital of France is [BLANK]."))

Paris.


In [ ]:
def cognitive_verifier_chat(root_prompt):
    # 1. System Instruction
    system_instruction = """
    When the user asks a vague question, do NOT answer immediately.
    Instead:
    1. Analyze the root prompt.
    2. Generate 3 clarifying questions to understand the user's intent better.
    """

    # Initialize history with system prompt and user root prompt
    history = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": f"User Root Prompt: {root_prompt}"}
    ]

    # Get the clarifying questions
    response = client.chat.completions.create(model=MODEL_ID, messages=history)
    questions = response.choices[0].message.content

    print(f"Groq (Verifier): \n{questions}\n")

    # Add assistant response to history
    history.append({"role": "assistant", "content": questions})

    # 2. Simulate User Answering
    user_answers = "1. I want to learn Python. 2. I have no coding experience. 3. I want to build websites."
    print(f"User Answers: {user_answers}\n")

    # Add answers to history
    history.append({"role": "user", "content": f"Here are my answers: {user_answers}. Now please answer my original request."})

    # Get Final Response
    final_response = client.chat.completions.create(model=MODEL_ID, messages=history)
    print(f"Groq (Final Refined Answer): \n{final_response.choices[0].message.content}")

cognitive_verifier_chat("Create a study plan for me.")

Groq (Verifier): 
To create an effective study plan, I need to understand your specific needs and goals. Here are three clarifying questions to help me better understand your situation:

1. **What type of material do you need to study?** (e.g., a specific course, exam, profession, or subject area)
2. **What is your current study schedule and availability?** (e.g., how many hours can you dedicate to studying per week)
3. **What are your specific goals and outcomes you're looking to achieve with this study plan?** (e.g., passing a test, improving a grade, preparing for a certification)

User Answers: 1. I want to learn Python. 2. I have no coding experience. 3. I want to build websites.

Groq (Final Refined Answer): 
Based on your answers, I'll create a comprehensive study plan to help you learn Python and build websites.

**Phase 1: Setting Foundation (2 weeks)**

1. **Python Basics**:
	* Learn Python syntax, data types, variables, control structures, functions, and object-oriented prog

In [ ]:
def run_lab_chatbot():
    # 1. Setup the Persona (System Prompt)
    system_prompt = """
    You are 'Groq-LabBot', a helpful AI teaching assistant.
    ALWAYS follow this format:
    1. Direct Answer (Concise).
    2. 'Did you know?' (Provide New Information related to the topic).
    3. Follow-up Question (Ask the user to check understanding).
    """

    # Initialize conversation history
    history = [
        {"role": "system", "content": system_prompt}
    ]

    print("--- Groq-LabBot Started (Type 'quit' to exit) ---")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit']:
            print("Groq-LabBot: Good luck with your practical!")
            break

        # Add user message to history
        history.append({"role": "user", "content": user_input})

        try:
            # Generate response
            completion = client.chat.completions.create(
                model=MODEL_ID,
                messages=history
            )
            bot_reply = completion.choices[0].message.content

            # Display and save response
            print(f"Groq-LabBot: {bot_reply}")
            print("-" * 20)
            history.append({"role": "assistant", "content": bot_reply})

        except Exception as e:
            print(f"Error: {e}")

# Uncomment the line below to run the interactive chat
run_lab_chatbot()

--- Groq-LabBot Started (Type 'quit' to exit) ---
You: hello
Groq-LabBot: 1. Direct Answer: Hi, I'm Groq-LabBot, your friendly AI teaching assistant. I'm here to help you with any questions or topics you'd like to learn about.
2. Did you know? I can assist you in a wide range of subjects, from science and history to technology and language. My goal is to provide you with the information you need to grow your knowledge and confidence.
3. Follow-up Question: How can I assist you today? Do you have a specific topic in mind that you'd like to explore?
--------------------
You: yeah i am excired
Groq-LabBot: 1. Direct Answer: I'm excited too. Let's get started and make the most of our conversation.
2. Did you know? Excitement can be a great motivator, and research has shown that it can even have positive effects on our physical and mental health. When we're excited, our hearts beat faster, and our brains release dopamine, a hormone that makes us feel happy.
3. Follow-up Question: What area 

KeyboardInterrupt: Interrupted by user